In [260]:
import pandas as pd
import numpy as np

In [267]:
df = pd.read_csv("data_mgz_attributes__24062023__1000_GMT3.csv", delimiter=';')
df2 = pd.read_csv('dataset_hackaton_ksg__v2__23062023__1710_GMT3.csv', delimiter=';')

C:\Users\sokol\AppData\Local\Temp\ipykernel_9856\3179656611.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('dataset_hackaton_ksg__v2__23062023__1710_GMT3.csv', delimiter=';')


In [269]:
df = df.drop(columns=['Unnamed: 0'])

In [270]:
df.columns = ['obj_key', 'состояние площадки', 'Площадь', 'Генпроектировщик',
       'Генподрядчик', 'Кол-во рабочих', 'date_report']

In [33]:
df = df.drop_duplicates(subset=['obj_key'])

In [242]:
df3 = pd.merge(df, df2, on='obj_key', how='right')

In [243]:
df3 = df3.drop(columns=['date_report_y', 'НазваниеЗадачи', '№ п/п', 'ПроцентЗавершенияЗадачи', ])
df3 = df3[df3['obj_key'].notna()]

In [99]:
df3

,obj_key,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,date_report_x,Unnamed: 0,obj_prg,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза
0,022-0527,Не передана,9850.0,0.0,0.0,NaN,2023.01.20,0,Образование,Общеобразовательные учреждения,1,2022-01-14,2023-03-30,NaN,NaN,NaN,NaN
1,022-0354,NaN,20271.0,0.0,0.0,NaN,2023.01.20,1,Образование,Общеобразовательные учреждения,1,2022-12-01,2023-07-27,NaN,NaN,NaN,NaN
2,022-0513,NaN,6892.0,0.0,0.0,NaN,2023.01.20,2,Образование,Дошкольные учреждения,1,2023-01-23,2023-09-29,NaN,NaN,NaN,NaN
3,020-0684,Занята,2488.6,1.0,2.0,NaN,2023.01.20,3,Образование,Дошкольные учреждения,1,2020-11-03,2022-02-01,2020-11-03,2021-12-29,NaN,NaN
4,019-0589,Занята,3751.1,1.0,2.0,NaN,2023.01.20,4,Образование,Дошкольные учреждения,1,2020-11-03,2022-05-16,2020-11-03,2021-12-29,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060160,023-0135,NaN,NaN,0.0,0.0,NaN,2023.03.15,63504,Здравоохранение,Поликлиники,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060161,021-0049,NaN,NaN,0.0,0.0,NaN,2023.01.20,63505,Здравоохранение,Поликлиники,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060162,021-0050,NaN,NaN,0.0,0.0,NaN,2023.01.20,63506,Здравоохранение,Поликлиники,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060163,022-0707,NaN,NaN,0.0,0.0,NaN,2023.01.20,63507,Здравоохранение,Поликлиники,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
df3['step'] = df3['obj_key'] + df3['Кодзадачи'].astype('str').apply(lambda x: x.split('.')[0] + x.split('.')[1] if x.count('.') >= 1 else x)
df3['ДатаНачалаЗадачи'] = pd.to_datetime(df3['ДатаНачалаЗадачи'], format="%Y-%m-%d")
df3['ДатаОкончанияЗадачи'] = pd.to_datetime(df3['ДатаОкончанияЗадачи'], format="%Y-%m-%d")
df3['time'] = (df3['ДатаОкончанияЗадачи'] - df3['ДатаНачалаЗадачи']).dt.days
df3['agreement'] = df3['ДатаначалаБП0'].isna()
df3['exp'] = df3['Экспертиза'].fillna('exp_none')
df3['year'] = df3['obj_key'].astype('str').apply(lambda x: 'year_' + x.split('-')[0])

In [245]:
df3 = df3.drop(columns=['date_report_x', 'Unnamed: 0', 'Статуспоэкспертизе', 'ДатаначалаБП0', 'ДатаокончанияБП0', 'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи'])

In [246]:
df3['Генпроектировщик'] = df3['Генпроектировщик'].apply(lambda x: 'GPR_' + str(x))
df3['Генподрядчик'] = df3['Генподрядчик'].apply(lambda x: 'GPD_' + str(x))
# df3['task'] = df3['Кодзадачи'].astype('str').apply(lambda x: x.split('.')[0] + x.split('.')[1] if x.count('.') >= 1 else x)
df3['task'] = df3['Кодзадачи'].astype('str')

In [249]:
df3

,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,obj_prg,obj_subprg,step,time,agreement,exp,year,task
0,9850.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-05271,440.0,True,exp_none,year_022,1
1,20271.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-03541,238.0,True,exp_none,year_022,1
2,6892.0,GPR_0.0,GPD_0.0,109.604173,Образование,Дошкольные учреждения,022-05131,249.0,True,exp_none,year_022,1
3,2488.6,GPR_1.0,GPD_2.0,109.604173,Образование,Дошкольные учреждения,020-06841,455.0,False,exp_none,year_020,1
4,3751.1,GPR_1.0,GPD_2.0,109.604173,Образование,Дошкольные учреждения,019-05891,559.0,False,exp_none,year_019,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1060148,13260.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-052296,0.0,False,exp_none,year_022,9.6.2
1060149,9850.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-052796,0.0,False,exp_none,year_022,9.6.2
1060150,18100.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-052696,0.0,False,exp_none,year_022,9.6.2
1060151,20271.0,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,022-035496,0.0,False,exp_none,year_022,9.6.2


In [201]:
df3 = df3.join(pd.get_dummies(df3['Генпроектировщик'])).join(pd.get_dummies(df3['Генподрядчик'])).join(pd.get_dummies(df3['obj_subprg'])).join(pd.get_dummies(df3['task'])).join(pd.get_dummies(df3['exp'])).join(pd.get_dummies(df3['obj_prg']))
df3 = df3.join(pd.get_dummies(df3['year']))
df3 = df3.drop(columns=['состояние площадки', 'Генпроектировщик', 'Экспертиза', 'exp', 'Генподрядчик', 'obj_subprg', 'Кодзадачи', 'obj_key', 'obj_prg', 'task', 'year'])
# .join(pd.get_dummies(df3['состояние площадки']))

In [247]:
df3 = df3.drop(columns=['состояние площадки', 'Экспертиза', 'obj_key', 'Кодзадачи'])
# .join(pd.get_dummies(df3['состояние площадки']))

In [248]:
df3['Кол-во рабочих'] = df3['Кол-во рабочих'].fillna(df3['Кол-во рабочих'].mean())
df3 = df3.dropna()

In [250]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [251]:
features = df3.drop(columns=['time', 'step'])
Time = df3['time']

In [252]:
df3.columns

Index(['Площадь', 'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих',
       'obj_prg', 'obj_subprg', 'step', 'time', 'agreement', 'exp', 'year',
       'task'],
      dtype='object')

In [253]:
x_train, x_test, y_train, y_test = train_test_split(features, Time, train_size=0.9, random_state=2)

In [95]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression().fit(x_train, y_train)

In [96]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mean_absolute_error(y_test, regr.predict(x_test))

62.37378870607145

In [50]:
# import autokeras
# model = autokeras.StructuredDataRegressor(max_trials=4,)
# model.fit(x_train, y_train, validation_data=(x_test, y_test))

In [236]:
x_train

,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,obj_prg,obj_subprg,agreement,exp,year,task
385576,6848.84,GPR_27.0,GPD_1.0,98.000000,Образование,Общеобразовательные учреждения,False,exp_none,year_020,3.15.7.3
525155,16484.00,GPR_8.0,GPD_5.0,109.604173,Образование,Общеобразовательные учреждения,False,exp_none,year_020,5.2.7.2.12
512367,11507.82,GPR_14.0,GPD_10.0,0.000000,Здравоохранение,Поликлиники,False,exp_none,year_019,4.5.2
432460,17488.00,GPR_1.0,GPD_2.0,109.604173,Образование,Общеобразовательные учреждения,False,exp_none,year_019,3.12.2.1
632616,4624.30,GPR_10.0,GPD_8.0,20.000000,Здравоохранение,Поликлиники,False,exp_none,year_019,4.9.0
...,...,...,...,...,...,...,...,...,...,...
252512,7515.86,GPR_27.0,GPD_1.0,33.000000,Образование,Общеобразовательные учреждения,False,exp_none,year_020,7.1.1.71
557490,1977.50,GPR_2.0,GPD_0.0,109.604173,Образование,Дошкольные учреждения,False,exp_none,year_021,3.5.2.3.2
961800,11410.00,GPR_11.0,GPD_7.0,65.000000,Образование,Общеобразовательные учреждения,False,exp_none,year_019,3.15.6.16
203246,9850.00,GPR_0.0,GPD_0.0,109.604173,Образование,Общеобразовательные учреждения,True,exp_none,year_022,1.2.1


In [109]:
import keras

In [254]:
from sklearn.metrics import mean_absolute_error

In [255]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(iterations=800,
                          loss_function='MAE',
                          eval_metric='MAE',
                          metric_period=50,
                        cat_features=['Генпроектировщик', 'Генподрядчик',
       'obj_prg', 'obj_subprg', 'agreement', 'exp', 'year', 'task']
                         )

In [256]:
cat.fit(x_train,y_train,verbose=False, plot=False)

In [282]:
mean_absolute_error(y_test, cat.predict(x_test))

44.39719663559547

In [258]:
cat.predict(x_test)

array([-2.02096228e-07,  1.01621764e+02,  9.00363810e-02, ...,
        1.01013565e+02,  6.53401805e+01,  5.82746145e+01])

In [280]:
cat.save_model('model', 'cbm')

In [262]:
test = pd.read_excel('Тест.xlsx')

In [297]:
 test.columns

Index(['№ п/п', 'obj_prg', 'obj_subprg', 'obj_key', 'obj_shortName',
       'Кодзадачи', 'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи',
       'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи', 'ДатаначалаБП0',
       'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза'],
      dtype='object')

In [313]:
dft = test
dft.drop(columns=['obj_shortName', 'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи'])

,№ п/п,obj_prg,obj_subprg,obj_key,Кодзадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза
0,36,Образование,Дошкольные учреждения,020-0684,1,2020-11-03,2022-02-01,2020-11-03,2021-12-29,NaN,NaN
1,35,Образование,Дошкольные учреждения,019-0589,1,2020-11-03,2022-05-16,2020-11-03,2021-12-29,NaN,NaN
2,61,Образование,Общеобразовательные учреждения,019-0594,1,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN
3,89,Культура,Культурные центры,021-0458,1,2021-10-12,2023-05-29,2021-10-12,2023-05-12,NaN,NaN
4,34,Образование,Дошкольные учреждения,017-0520,1,2018-11-01,2022-02-15,2018-11-01,2022-02-15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
64659,21,Здравоохранение,Поликлиники,023-0135,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64660,28,Здравоохранение,Поликлиники,021-0049,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64661,29,Здравоохранение,Поликлиники,021-0050,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64662,31,Здравоохранение,Поликлиники,022-0707,NaN,NaT,NaT,NaT,NaT,NaN,NaN


In [309]:
df.drop(columns=['date_report'])
df = df.drop_duplicates(subset=['obj_key'])

'площадь', 'Генпроектировщик', 'Генподрядчик', 'кол-во рабочих',
       'obj_prg', 'obj_subprg', 'agreement', 'exp', 'year', 'task'

In [317]:
dft = dft.merge(df, on='obj_key', how='left')

In [318]:
dft

,№ п/п,obj_prg,obj_subprg,obj_key,obj_shortName,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,...,time,agreement,exp,year,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,date_report
0,36,Образование,Дошкольные учреждения,020-0684,"ДОУ на 125, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-02-01,...,455.0,False,exp_none,year_020,Занята,2488.6,1.0,2.0,NaN,2023.01.20
1,35,Образование,Дошкольные учреждения,019-0589,"ДОУ на 225, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-05-16,...,559.0,False,exp_none,year_019,Занята,3751.1,1.0,2.0,NaN,2023.01.20
2,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,...,225.0,False,exp_none,year_019,Занята,17488.0,1.0,2.0,NaN,2023.01.20
3,89,Культура,Культурные центры,021-0458,"КСЦ ""Печатники"", Полбина",1,Предпроектные работы,100.0,2021-10-12,2023-05-29,...,594.0,False,exp_none,year_021,NaN,NaN,NaN,NaN,NaN,NaN
4,34,Образование,Дошкольные учреждения,017-0520,"ДОУ на 350, ул. 6-я Радиальная и ул. Дуговая",1,Предпроектные работы,100.0,2018-11-01,2022-02-15,...,1202.0,False,exp_none,year_017,"Свободна, не передана",6367.0,8.0,5.0,NaN,2023.01.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64659,21,Здравоохранение,Поликлиники,023-0135,"ДП на 320, ул. Зеленоградская, 2 этап. Ликвид...",NaN,NaN,NaN,NaT,NaT,...,NaN,True,exp_none,year_023,NaN,NaN,0.0,0.0,NaN,2023.03.15
64660,28,Здравоохранение,Поликлиники,021-0049,Пристройка к поликлинике ул. Кременчугская,NaN,NaN,NaN,NaT,NaT,...,NaN,True,exp_none,year_021,NaN,NaN,0.0,0.0,NaN,2023.01.20
64661,29,Здравоохранение,Поликлиники,021-0050,"ДВП на 560, Проектируемый проезд 3538",NaN,NaN,NaN,NaT,NaT,...,NaN,True,exp_none,year_021,NaN,NaN,0.0,0.0,NaN,2023.01.20
64662,31,Здравоохранение,Поликлиники,022-0707,"ДВП на 320, Онежская",NaN,NaN,NaN,NaT,NaT,...,NaN,True,exp_none,year_022,NaN,NaN,0.0,0.0,NaN,2023.01.20


In [319]:
dft['ДатаНачалаЗадачи'] = pd.to_datetime(dft['ДатаНачалаЗадачи'], format="%Y-%m-%d")
dft['ДатаОкончанияЗадачи'] = pd.to_datetime(dft['ДатаОкончанияЗадачи'], format="%Y-%m-%d")
dft['time'] = (dft['ДатаОкончанияЗадачи'] - dft['ДатаНачалаЗадачи']).dt.days
dft['agreement'] = dft['ДатаначалаБП0'].isna()
dft['exp'] = dft['Экспертиза'].fillna('exp_none')
dft['year'] = dft['obj_key'].astype('str').apply(lambda x: 'year_' + x.split('-')[0])
dft['Генпроектировщик'] = dft['Генпроектировщик'].apply(lambda x: 'GPR_' + str(x))
dft['Генподрядчик'] = dft['Генподрядчик'].apply(lambda x: 'GPD_' + str(x))
dft['task'] = dft['Кодзадачи'].astype('str')
dft['Кол-во рабочих'] = dft['Кол-во рабочих'].fillna(dft['Кол-во рабочих'].mean())
dft = dft.fillna(0)

In [320]:
dft

,№ п/п,obj_prg,obj_subprg,obj_key,obj_shortName,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,...,agreement,exp,year,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,date_report,task
0,36,Образование,Дошкольные учреждения,020-0684,"ДОУ на 125, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03 00:00:00,2022-02-01 00:00:00,...,False,exp_none,year_020,Занята,2488.6,GPR_1.0,GPD_2.0,109.601623,2023.01.20,1
1,35,Образование,Дошкольные учреждения,019-0589,"ДОУ на 225, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03 00:00:00,2022-05-16 00:00:00,...,False,exp_none,year_019,Занята,3751.1,GPR_1.0,GPD_2.0,109.601623,2023.01.20,1
2,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04 00:00:00,2021-12-15 00:00:00,...,False,exp_none,year_019,Занята,17488.0,GPR_1.0,GPD_2.0,109.601623,2023.01.20,1
3,89,Культура,Культурные центры,021-0458,"КСЦ ""Печатники"", Полбина",1,Предпроектные работы,100.0,2021-10-12 00:00:00,2023-05-29 00:00:00,...,False,exp_none,year_021,0,0.0,GPR_nan,GPD_nan,109.601623,0,1
4,34,Образование,Дошкольные учреждения,017-0520,"ДОУ на 350, ул. 6-я Радиальная и ул. Дуговая",1,Предпроектные работы,100.0,2018-11-01 00:00:00,2022-02-15 00:00:00,...,False,exp_none,year_017,"Свободна, не передана",6367.0,GPR_8.0,GPD_5.0,109.601623,2023.01.20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64659,21,Здравоохранение,Поликлиники,023-0135,"ДП на 320, ул. Зеленоградская, 2 этап. Ликвид...",0,0,0.0,0,0,...,True,exp_none,year_023,0,0.0,GPR_0.0,GPD_0.0,109.601623,2023.03.15,nan
64660,28,Здравоохранение,Поликлиники,021-0049,Пристройка к поликлинике ул. Кременчугская,0,0,0.0,0,0,...,True,exp_none,year_021,0,0.0,GPR_0.0,GPD_0.0,109.601623,2023.01.20,nan
64661,29,Здравоохранение,Поликлиники,021-0050,"ДВП на 560, Проектируемый проезд 3538",0,0,0.0,0,0,...,True,exp_none,year_021,0,0.0,GPR_0.0,GPD_0.0,109.601623,2023.01.20,nan
64662,31,Здравоохранение,Поликлиники,022-0707,"ДВП на 320, Онежская",0,0,0.0,0,0,...,True,exp_none,year_022,0,0.0,GPR_0.0,GPD_0.0,109.601623,2023.01.20,nan


In [323]:
dft = dft.drop(columns=['№ п/п', 'obj_key', 'obj_shortName',
       'Кодзадачи', 'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи',
       'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи', 'ДатаначалаБП0',
       'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза', 'состояние площадки'])

In [328]:
dft = dft.drop(columns=['date_report'])

In [333]:
dft.columns

Index(['obj_prg', 'obj_subprg', 'time', 'agreement', 'exp', 'year', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих', 'task'],
      dtype='object')

In [336]:
dft = dft[['Площадь','Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих', 'obj_prg', 'obj_subprg', 'agreement', 'exp', 'year', 'task']]

'площадь', 'Генпроектировщик', 'Генподрядчик', 'кол-во рабочих',
       'obj_prg', 'obj_subprg', 'agreement', 'exp', 'year', 'task'

In [337]:
dft

,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,obj_prg,obj_subprg,agreement,exp,year,task
0,2488.6,GPR_1.0,GPD_2.0,109.601623,Образование,Дошкольные учреждения,False,exp_none,year_020,1
1,3751.1,GPR_1.0,GPD_2.0,109.601623,Образование,Дошкольные учреждения,False,exp_none,year_019,1
2,17488.0,GPR_1.0,GPD_2.0,109.601623,Образование,Общеобразовательные учреждения,False,exp_none,year_019,1
3,0.0,GPR_nan,GPD_nan,109.601623,Культура,Культурные центры,False,exp_none,year_021,1
4,6367.0,GPR_8.0,GPD_5.0,109.601623,Образование,Дошкольные учреждения,False,exp_none,year_017,1
...,...,...,...,...,...,...,...,...,...,...
64659,0.0,GPR_0.0,GPD_0.0,109.601623,Здравоохранение,Поликлиники,True,exp_none,year_023,nan
64660,0.0,GPR_0.0,GPD_0.0,109.601623,Здравоохранение,Поликлиники,True,exp_none,year_021,nan
64661,0.0,GPR_0.0,GPD_0.0,109.601623,Здравоохранение,Поликлиники,True,exp_none,year_021,nan
64662,0.0,GPR_0.0,GPD_0.0,109.601623,Здравоохранение,Поликлиники,True,exp_none,year_022,nan


In [343]:
predicts = cat.predict(dft)

In [339]:
tt = test

In [340]:
tt.columns

Index(['№ п/п', 'obj_prg', 'obj_subprg', 'obj_key', 'obj_shortName',
       'Кодзадачи', 'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи',
       'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи', 'ДатаначалаБП0',
       'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза', 'time',
       'agreement', 'exp', 'year'],
      dtype='object')

In [341]:
finish = test[['obj_key', 'Кодзадачи', 'НазваниеЗадачи']]

In [354]:
predictions_df = pd.DataFrame(predicts.tolist(), columns=['time in days'])

In [355]:
predictions_df

,time in days
0,133.535306
1,147.136815
2,151.986888
3,220.390681
4,140.941285
...,...
64659,22.867059
64660,23.308048
64661,23.308048
64662,23.193689


In [360]:
finish['time in days'] = predicts

C:\Users\sokol\AppData\Local\Temp\ipykernel_9856\2748903942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finish['time in days'] = predicts


In [367]:
finish['сдвиг'] = (test['ДатаОкончанияЗадачи'] - test['ДатаНачалаЗадачи']).dt.days

C:\Users\sokol\AppData\Local\Temp\ipykernel_9856\3175304259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finish['сдвиг'] = (test['ДатаОкончанияЗадачи'] - test['ДатаНачалаЗадачи']).dt.days


In [371]:
finish['сдвиг'] = finish['time in days'] - finish['сдвиг']

C:\Users\sokol\AppData\Local\Temp\ipykernel_9856\2562606441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finish['сдвиг'] = finish['time in days'] - finish['сдвиг']


In [372]:
finish

,obj_key,Кодзадачи,НазваниеЗадачи,time in days,сдвиг
0,020-0684,1,Предпроектные работы,133.535306,-321.464694
1,019-0589,1,Предпроектные работы,147.136815,-411.863185
2,019-0594,1,Предпроектные работы,151.986888,-73.013112
3,021-0458,1,Предпроектные работы,220.390681,-373.609319
4,017-0520,1,Предпроектные работы,140.941285,-1061.058715
...,...,...,...,...,...
64659,023-0135,NaN,NaN,22.867059,NaN
64660,021-0049,NaN,NaN,23.308048,NaN
64661,021-0050,NaN,NaN,23.308048,NaN
64662,022-0707,NaN,NaN,23.193689,NaN


In [374]:
finish.to_excel('test_predictions.xlsx')